In [1]:
import numpy as np
import tensorflow as tf
import os
from scipy.special import softmax

In [2]:
import random

seed = 39487
print("Seed: ", seed)
# tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED']=str(seed)

Seed:  39487


In [3]:
relu = lambda x: x * (x > 0)

In [4]:
def block_matvec_mul(mat_A, vec_B, BLOCK_WIDTH, BLOCK_HEIGHT, error_profile):
    gridydim = int(np.ceil(mat_A.shape[0]/BLOCK_HEIGHT))
    gridxdim = int(np.ceil(mat_A.shape[1]/BLOCK_WIDTH))
#     print("GRID: (",gridydim,",",gridxdim,")")

    block_result = np.zeros((gridydim, gridxdim,BLOCK_HEIGHT ))
    block_error_profile = error_profile.reshape(gridydim, gridxdim, BLOCK_HEIGHT)
    for x in range(gridxdim):
        for y in range(gridydim):
            block_result[y][x] = np.matmul(mat_A[y*BLOCK_HEIGHT:(y+1)*BLOCK_HEIGHT,
                                                 x*BLOCK_WIDTH:(x+1)*BLOCK_WIDTH], vec_B[x*BLOCK_WIDTH:(x+1)*BLOCK_WIDTH]).squeeze()
            block_result[y][x] += block_error_profile[y][x]

    # sum columnwise i.e., sum elements in each row i.e., axis = 1
    block_sum = block_result.sum(axis=1)

    #flatten to single vector
    product = block_sum.reshape(-1,1)

    return product

In [5]:
def manual_inference(input_image, error_profile):
    # flatten input

    flattened_input = input_image.reshape(-1,1)

    # send through Layer_0
    l0_BLOCK_WIDTH  = 98
    l0_BLOCK_HEIGHT = 64

    l0_NO_OF_SM_X = 8 # l0_BLOCK_WIDTH * l0_NO_OF_SM_X = l0_INPUT_LENGHT (784)
    l0_NO_OF_SM_Y = 8 # l0_BLOCK_HEIGHT * l0_NO_OF_SM_Y = l0_LAYER_WIDTH (512)
    l0_TOTAL_SM = l0_NO_OF_SM_X * l0_NO_OF_SM_Y
    l0_NO_OF_THREADS_PER_SM = l0_BLOCK_HEIGHT
    l0_error_profile = error_profile[0:l0_TOTAL_SM,
                                     0:l0_NO_OF_THREADS_PER_SM]

    # matvec multiplication
    # l0 = np.matmul(dense_weights.transpose(), 
    #                flattened_input)
    l0 = block_matvec_mul(dense_weights.transpose(),
                          flattened_input,
                          BLOCK_WIDTH=l0_BLOCK_WIDTH,
                          BLOCK_HEIGHT=l0_BLOCK_HEIGHT,
                          error_profile=l0_error_profile)

    # add bias
    b0 = l0 + dense_biases

    # ReLU
    b0_relu = relu(b0)

    # send through Layer_1
    l1_BLOCK_WIDTH  = 64
    l1_BLOCK_HEIGHT = 64

    l1_NO_OF_SM_X = 8 # l1_BLOCK_WIDTH * l1_NO_OF_SM_X = l1_INPUT_LENGHT (784)
    l1_NO_OF_SM_Y = 8 # l1_BLOCK_HEIGHT * l1_NO_OF_SM_Y = l1_LAYER_WIDTH (512)
    l1_TOTAL_SM = l1_NO_OF_SM_X * l1_NO_OF_SM_Y
    l1_NO_OF_THREADS_PER_SM = l1_BLOCK_HEIGHT
    l1_error_profile = error_profile[0:l1_TOTAL_SM,
                                     0:l1_NO_OF_THREADS_PER_SM]

    # matvec multiplication
    # l1 = np.matmul(dense_1_weights.transpose(), 
    #                b0_relu)
    l1 = block_matvec_mul(dense_1_weights.transpose(),
                          b0_relu,
                          BLOCK_WIDTH=l1_BLOCK_WIDTH,
                          BLOCK_HEIGHT=l1_BLOCK_HEIGHT,
                          error_profile=l1_error_profile)

    # add bias
    b1 = l1 + dense_1_biases

    # ReLU
    b1_relu = relu(b1)

    # send through Layer_2
    l2_BLOCK_WIDTH  = 64
    l2_BLOCK_HEIGHT = 10

    l2_NO_OF_SM_X = 8 # l2_BLOCK_WIDTH * l2_NO_OF_SM_X = l2_INPUT_LENGHT (784)
    l2_NO_OF_SM_Y = 1 # l2_BLOCK_HEIGHT * l2_NO_OF_SM_Y = l2_LAYER_WIDTH (512)
    l2_TOTAL_SM = l2_NO_OF_SM_X * l2_NO_OF_SM_Y
    l2_NO_OF_THREADS_PER_SM = l2_BLOCK_HEIGHT
    l2_error_profile = error_profile[0:l2_TOTAL_SM,
                                     0:l2_NO_OF_THREADS_PER_SM]
    # matvec multiplication
    # l2 = np.matmul(dense_2_weights.transpose(), 
    #                b1_relu)
    l2 = block_matvec_mul(dense_2_weights.transpose(), 
                         b1_relu,
                         BLOCK_WIDTH=64,
                         BLOCK_HEIGHT=10,
                         error_profile=l2_error_profile)

    # add bias
    b2 = l2 + dense_2_biases

    # softmax
    b2_softmax = softmax(b2)

    # Output
    prediction = np.argmax(b2_softmax)
#     print("PREDICTION: ", prediction)
    
    return prediction

In [6]:
# Combine test and train images together into one dataset

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0  

all_images =np.concatenate([train_images, test_images], axis=0)
all_labels =np.concatenate([train_labels, test_labels], axis=0)

In [7]:
# Arguments
layer_width = 512
model_seed = 52233264

In [8]:
# load weights
tag = "mnist_dense" + '-w' + str(layer_width) + 'x' + str(layer_width) + '-' + str(model_seed)
cur_folder = os.getcwd()
weights_folder = os.path.join(cur_folder,"weights",tag)
weight_filename_tag = os.path.join(weights_folder, tag + "_")

# Layer_0
dense_weights_file = weight_filename_tag + "dense_weights.npy"
dense_biases_file = weight_filename_tag + "dense_biases.npy"
dense_weights = np.load(dense_weights_file)
dense_biases = np.load(dense_biases_file).reshape(-1,1)

# Layer_1
dense_1_weights_file = weight_filename_tag + "dense_1_weights.npy"
dense_1_biases_file = weight_filename_tag + "dense_1_biases.npy"
dense_1_weights = np.load(dense_1_weights_file)
dense_1_biases = np.load(dense_1_biases_file).reshape(-1,1)

#Layer_2
dense_2_weights_file = weight_filename_tag + "dense_2_weights.npy"
dense_2_biases_file = weight_filename_tag + "dense_2_biases.npy"
dense_2_weights = np.load(dense_2_weights_file)
dense_2_biases = np.load(dense_2_biases_file).reshape(-1,1)

In [9]:
# # load input
# image_no = 69999;
# input_image = all_images[image_no]
# input_image_label = all_labels[image_no]
# # print("IMAGE_NUMBER: ", image_no)
# # print("TRUTH_LABEL: ", input_image_label)

In [10]:
# NO_OF_IMAGES = all_images.shape[0]
NO_OF_IMAGES = 20000

test_images = np.random.choice(all_images.shape[0], size=(NO_OF_IMAGES))

In [11]:
zero_error_profile = np.zeros((68,64))
correct_predictions = 0;
for image_no in test_images:
    if manual_inference(all_images[image_no],
                        zero_error_profile) == all_labels[image_no]:
        correct_predictions += 1
print("Accuracy: ", 100*correct_predictions/NO_OF_IMAGES)

Accuracy:  99.81


In [12]:
error_profile = np.load("./data/uniform_20210907.npy") #shape(68, 64)

In [13]:
correct_predictions = 0;
for image_no in test_images:
    if manual_inference(all_images[image_no],
                        error_profile) == all_labels[image_no]:
        correct_predictions += 1
print("Accuracy: ", 100*correct_predictions/NO_OF_IMAGES)

Accuracy:  22.67


In [14]:
for i in range(100):
    for row in error_profile:
        np.random.shuffle(row)


    correct_predictions = 0;
    for image_no in test_images:
        if manual_inference(all_images[image_no],
                            error_profile) == all_labels[image_no]:
            correct_predictions += 1
    print("Accuracy: ", 100*correct_predictions/NO_OF_IMAGES)

Accuracy:  23.295
Accuracy:  25.255
Accuracy:  27.975
Accuracy:  20.875
Accuracy:  20.335
Accuracy:  25.525
Accuracy:  24.435
Accuracy:  22.225
Accuracy:  27.25
Accuracy:  22.495
Accuracy:  24.87
Accuracy:  26.33
Accuracy:  23.475
Accuracy:  25.89
Accuracy:  24.075
Accuracy:  22.95
Accuracy:  21.05
Accuracy:  26.175
Accuracy:  24.45
Accuracy:  24.44
Accuracy:  25.945
Accuracy:  23.12
Accuracy:  21.035
Accuracy:  22.39
Accuracy:  25.34
Accuracy:  24.815
Accuracy:  25.05
Accuracy:  23.505
Accuracy:  24.39
Accuracy:  23.935
Accuracy:  20.82
Accuracy:  25.275
Accuracy:  21.405
Accuracy:  27.82
Accuracy:  18.6
Accuracy:  21.255
Accuracy:  25.595
Accuracy:  15.15
Accuracy:  25.87
Accuracy:  21.21
Accuracy:  20.755
Accuracy:  28.25
Accuracy:  21.61
Accuracy:  24.035
Accuracy:  24.73
Accuracy:  22.43
Accuracy:  27.53
Accuracy:  25.0
Accuracy:  27.14
Accuracy:  25.32
Accuracy:  16.545
Accuracy:  24.425
Accuracy:  24.515
Accuracy:  26.18
Accuracy:  22.145
Accuracy:  26.13
Accuracy:  23.435
Accur